In [1]:
import os
import pandas as pd

# Function to convert files (.xls, .xlsx, .html) to CSV
def convert_file_to_csv(file_path, csv_file_path):
    try:
        # Attempt to read the file as an Excel file, skipping the first two rows
        if file_path.endswith('.xls') or file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path, engine='openpyxl' if file_path.endswith('.xlsx') else 'xlrd', skiprows=2)
        else:
            # If it's a CSV, skip the first two rows
            df = pd.read_csv(file_path, skiprows=2)  # Skip the first 2 rows for CSV files

        # Save to CSV
        df.to_csv(csv_file_path, index=False)
        print(f"Converted {file_path} to {csv_file_path} as Excel or CSV")
    except Exception as e:
        print(f"Failed to convert {file_path}: {e}")
        
        # Try reading as an HTML file if it contains web page data
        try:
            df = pd.read_html(file_path)[0]  # Read the first table
            df.to_csv(csv_file_path, index=False)
            print(f"Converted {file_path} to {csv_file_path} as HTML")
        except Exception as e:
            print(f"Failed to convert {file_path} as HTML: {e}")

# Function to recursively search for files (.xls, .xlsx, .html) and convert them to CSV
def convert_files_to_csv(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check for .xls, .xlsx, .html extensions
            if file.endswith('.xls') or file.endswith('.xlsx') or file.endswith('.html') or file.endswith('.csv'):
                file_path = os.path.join(root, file)
                csv_file_path = os.path.join(root, f"{os.path.splitext(file)[0]}.csv")
                
                # Convert the file to CSV
                convert_file_to_csv(file_path, csv_file_path)

# Initialize an empty DataFrame to hold the merged data
merged_df = pd.DataFrame()

# Function to recursively search for CSV files and merge them
def merge_csv_files(directory):
    global merged_df
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is a CSV file (case insensitive)
            if file.lower().endswith('.csv'):
                file_path = os.path.join(root, file)
                try:
                    # Read the CSV file into a DataFrame, skipping the first two rows
                    df = pd.read_csv(file_path, skiprows=2)  # Skipping the first 2 rows when reading CSV files
                    if not df.empty:
                        # Append the DataFrame to the merged DataFrame
                        merged_df = pd.concat([merged_df, df], ignore_index=True)
                        print(f'Merged: {file_path}')
                    else:
                        print(f'Skipped empty file: {file_path}')
                except pd.errors.EmptyDataError:
                    print(f'Skipped file with empty data: {file_path}')
                except Exception as e:
                    print(f'Error processing file {file_path}: {e}')

# Function to perform the entire process of conversion and merging
def convert_and_merge_files(folder_path):
    # Convert all relevant files (.xls, .xlsx, .html) to CSV
    convert_files_to_csv(folder_path)
    
    # Merge all CSV files found in the folder
    merge_csv_files(folder_path)

    # Save the merged DataFrame to a new CSV file
    output_file_path = os.path.join(folder_path, '19062025.csv')
    merged_df.to_csv(output_file_path, index=False)

    print(f"All CSV files have been merged into '{output_file_path}'")

# Define the folder path (use raw string to avoid issues with backslashes)
folder_path = r'E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File'  # Update with your folder path

# Perform the conversion and merging process
convert_and_merge_files(folder_path)

Failed to convert E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\DAYWISE_890_20062025_12020245071002100953_YASH.xls: Pandas requires version '2.0.1' or newer of 'xlrd' (version '1.2.0' currently installed).
Converted E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\DAYWISE_890_20062025_12020245071002100953_YASH.xls to E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\DAYWISE_890_20062025_12020245071002100953_YASH.csv as HTML
Merged: E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\DAYWISE_890_20062025_12020245071002100953_YASH.csv
All CSV files have been merged into 'E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\19062025.csv'


In [2]:
import os
import pandas as pd

def process_csv_files(folder_path):
    all_data = []  # List to hold all dataframes

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                # Extract the date from the filename (remove the .csv extension)
                date_value = file[:-4]  # e.g., '01042024'
                
                # Convert to datetime using the correct format (DDMMYYYY)
                formatted_date = pd.to_datetime(date_value, format='%d%m%Y').strftime('%d-%m-%Y')
                
                file_path = os.path.join(root, file)

                # Read the CSV file (without skipping any rows)
                df = pd.read_csv(file_path)
                
                # Add the TradeDate column with the formatted date
                df['TradeDate'] = formatted_date
                
                # Append the modified DataFrame to the list
                all_data.append(df)

    # Combine all dataframes into one
    combined_df = pd.concat(all_data, ignore_index=True)

    # Save the combined DataFrame to a new CSV file 
    combined_df.to_csv(os.path.join(folder_path, '19.csv'), index=False)

# Specify the path to your main folder
main_folder = r'E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File'  # or use forward slashes
process_csv_files(main_folder)


In [3]:
import pandas as pd

# Load the CSV file
df = pd.read_csv(r'E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\19.csv')

# Check the data type of 'TradeDate'
print("Data type of 'TradeDate':", df['TradeDate'].dtype)

# If 'TradeDate' is not a string, convert it to string first
if not pd.api.types.is_string_dtype(df['TradeDate']):
    df['TradeDate'] = df['TradeDate'].astype(str)

# Now convert 'TradeDate' to datetime format
try:
    df['TradeDate'] = pd.to_datetime(df['TradeDate'].str.replace('/', '-'), format='%d-%m-%Y')
except KeyError as e:
    print(f"KeyError: {e}")
except Exception as e:
    print(f"Error: {e}")

# Display the updated DataFrame (optional, you can remove this if not needed)
print("Updated DataFrame:\n", df.head())

# Save the updated DataFrame back to the same CSV fil
df.to_csv(r'E:\Back-Up Files\Data Team\2024-2025\ADP\Current Date File\19.csv', index=False)

print("The date format has been updated and saved back to the same file.")


Data type of 'TradeDate': object
Updated DataFrame:
   CLIENT_ID COMPANY_CODE                 SCRIP_SYMBOL  NET_QUANTITY  NET_RATE  \
0  ADPALL01  DERIVATIVES  NIFTY 19Jun25 CE 24800.0000             0    0.0000   
1  ADPALL01  DERIVATIVES  NIFTY 19Jun25 CE 24900.0000             0    0.0000   
2  ADPALL01  DERIVATIVES  NIFTY 19Jun25 PE 24800.0000             0    0.0000   
3  ADPALL01  DERIVATIVES  NIFTY 19Jun25 PE 24900.0000             0    0.0000   
4  ADPALL01  DERIVATIVES  NIFTY 26Jun25 CE 24900.0000          2625  151.4114   

   NET_AMOUNT  CLOSING_PRICE  NOT_PROFIT  TRADING_QUANTITY  TRADING_AMOUNT  \
0    49541.25           0.00    49541.25                 0            0.00   
1   -55875.00           0.00   -55875.00              1950           42.45   
2    -8313.75           0.00    -8313.75                 0            0.00   
3    83718.75           0.00    83718.75             -1950          142.70   
4  -397455.00         142.05   -24573.75                 0            

In [4]:
import os
import pandas as pd 

# Path to the main folder
main_folder_path = r'E:\Back-Up Files\Data Team\2024-2025\ADP\CLUB'  # Use raw string literal to handle backslashes

# Initialize an empty DataFrame to hold the merged data
merged_df = pd.DataFrame() 

# Function to recursively search for CSV files and merge them
def merge_csv_files(directory):
    global merged_df
    # Walk through the directory tree
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is a CSV file (case insensitive)
            if file.lower().endswith('.csv'):
                # Construct the full path to the CSV file
                file_path = os.path.join(root, file)
                try:
                    # Read the CSV file into a DataFrame
                    df = pd.read_csv(file_path)
                    if not df.empty:  # Check if DataFrame is not empty
                        # Append the DataFrame to the merged DataFrame
                        merged_df = pd.concat([merged_df, df], ignore_index=True)
                        print(f'Merged: {file_path}')  # Optional: print each file being merged
                    else:
                        print(f'Skipped empty file: {file_path}')
                except pd.errors.EmptyDataError:
                    print(f'Skipped file with empty data: {file_path}')
                except Exception as e:
                    print(f'Error processing file {file_path}: {e}') 

# Call the function to start merging CSV files
merge_csv_files(main_folder_path) 

# Save the merged DataFrame to a new CSV file
output_file_path = os.path.join(main_folder_path, '25_ClubADP_19.csv')
merged_df.to_csv(output_file_path, index=False) 

print(f"All CSV files have been merged into '{output_file_path}'")


Merged: E:\Back-Up Files\Data Team\2024-2025\ADP\CLUB\19.csv
Merged: E:\Back-Up Files\Data Team\2024-2025\ADP\CLUB\25_ClubADP_18.csv
All CSV files have been merged into 'E:\Back-Up Files\Data Team\2024-2025\ADP\CLUB\25_ClubADP_19.csv'
